# Write & Rewrite Barium forcing data in NEMO

In [2]:
import glob
import scipy.io
import numpy as np
import netCDF4 as nc
from __future__ import print_function
% matplotlib inline

# Barium river source & boundary

In [17]:
Ba_name=glob.glob('_data/Exchange/Ba_ORCA2_2deg.mat')
Ba_Mat=scipy.io.loadmat(Ba_name[0])
Ba_ORCA2_2deg=Ba_Mat['Ba_ORCA2_2deg']
nav_lon=Ba_Mat['nav_lon']
nav_lat=Ba_Mat['nav_lat']

In [3]:
boundary_name=glob.glob('_data/Exchange/Ba_boundary.mat')
Boundary_Mat=scipy.io.loadmat(boundary_name[0])
Ba_boundary=Boundary_Mat['Ba_boundary']

In [4]:
Ba_boundary.shape

(149L, 182L)

** Get `time_counter` from Xianmin's original file. **

In [5]:
ANHA4_name=glob.glob('_data/NEMO/ANHA4_runoff_monthly_DaiTrenberth_fewPTs.nc')
ANH4_obj=nc.Dataset(ANHA4_name[0])
time_counter=ANH4_obj.variables['time_counter'][:]

In [6]:
print(time_counter)

[  15.   46.   75.  106.  136.  167.  197.  228.  259.  289.  320.  350.]


In [7]:
nc_obj = nc.Dataset('_data/Exchange/Ba_ORCA2_2deg_test.nc', 'w') # format='NETCDF4'
nc_obj.description = 'River sources of Barium concentration in Arctic'
print(nc_obj.file_format)

NETCDF4


In [8]:
Ba_ORCA2_2deg.shape

(12L, 149L, 182L)

In [9]:
time=nc_obj.createDimension('time', None)
lat=nc_obj.createDimension('lat', 149)
lon=nc_obj.createDimension('lon', 182)

In [10]:
print(time.isunlimited())

True


In [11]:
time_counter_obj=nc_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=nc_obj.createVariable('Ba_ORCA2_2deg', np.float64, ('time', 'lat', 'lon'), zlib=True)
BaB_obj = nc_obj.createVariable('Ba_boundary', np.float64, ('lat', 'lon'), zlib=True)
nav_lat_obj=nc_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=nc_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [12]:
time_counter_obj[:]=time_counter
BaVar_obj[:]=Ba_ORCA2_2deg
BaB_obj[:]=Ba_boundary
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon

In [13]:
BaVar_obj.units='1e-6 mol/L'
#BaVar_obj._Fillvalue = 0.
#BaVar_obj.add_offset = 0.

In [14]:
nc_obj.close()

#Barium Initial field

In [15]:
MAT = scipy.io.loadmat('_data/Exchange/NEMO_ORCA2_Ba.mat')
Ba_orca = MAT['Ba_ini_orca'][:]
#Ba_orca[np.isnan(Ba_orca)] = -999

In [16]:
ini_obj = nc.Dataset('_data/Exchange/Ba_ORCA2_Nomask.nc', 'w') # format='NETCDF4'
ini_obj.description = 'Barium Initial Field'
print(ini_obj.file_format)

NETCDF4


In [17]:
time=ini_obj.createDimension('time', None)
lev = ini_obj.createDimension('lev', 31)
lat=ini_obj.createDimension('lat', 149)
lon=ini_obj.createDimension('lon', 182)

In [18]:
time_counter_obj=ini_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=ini_obj.createVariable('Ba', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
nav_lat_obj=ini_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=ini_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [19]:
Ba = np.empty([1, 31, 149, 182])
Ba[0, :, :, :] = Ba_orca

In [20]:
time_counter_obj[:]=3600.0
BaVar_obj[:]=Ba
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon

In [21]:
BaVar_obj.units='1e-6 mol/L'

In [22]:
ini_obj.close()